In [70]:
import random
# deterministic finite state automaton
class DFSA:
    
    def __init__(self, n, auto_fill = True, initial = None, final = None, not_final = None, states = None):
        #setting the dfsa details
        self.n = n
        self.alphabet = ['a','b']
        self.visited = [] #list of state ids
        
        if auto_fill == True:
            #randomising the initial state
            self.initial = random.randint(0, n-1) #choosing a random state as the initial state
            self.final = [] #list of state ids
            self.not_final = [] #list of state ids
            self.states = [] #list of states
            
            for i in range(n):
                #creating states and appending them to the states array
                state = State(s_id=i)
                self.states.append(state)
                #appending the state ids to either the final or not final arrays
                if state.accepting == True:
                    self.final.append(i)
                elif state.accepting == False:
                    self.not_final.append(i)
        
            #sorting the final and not final arrays
            self.final.sort()
            self.not_final.sort()
            
            #randomly setting the state transitions
            for i in range(n):
                a = random.randint(0,n-1) #choosing random states for transition a
                b = random.randint(0,n-1) #choosing random states for transition b
                self.states[i].set_transitions(a=a,b=b)
                
        elif auto_fill == False:
            #filling the dfsa with the given details
            self.initial = initial
            self.final = final
            self.not_final = not_final
            self.states = states
            
        
        
    def print_details(self):
        #printing the dfsa details
        print()
        print("Number of states: ", self.n)
        print("Alphabet: ", self.alphabet)
        print("Initial state id: ", self.initial)
        print("Final states array: ", self.final)
        print("Not final states array: ",self.not_final)
        print()
        for state in self.states:
            state.print_details()

    
    def breadth_first_search(self):
        max_depth = 0
        visited = [False] * (len(self.states))
        queue = []
        
        #storing the depth of each node from initial state 
        depth = {i: 0 for i in range(self.n)}
        #setting the depth for the initial state to be 0
        depth [0] = 0
        
        #starting from the initial state
        #adding initial state to queue
        queue.append(self.initial)
        visited[self.initial] = True
        
        #looping whilst queue isn't empty
        while queue:
            #popping the value from the queue
            current = queue.pop(0)
            #getting the state ids stored as the transitions of the current state
            for s_id in self.states[current].transitions.values():
                #if state has not already been seen
                if visited[s_id] == False:
                    #appending the transition state id to the queue and to the visited array
                    queue.append(s_id)
                    print("Current: ", current, "\nPushed: ",s_id, "\n")
                    visited[s_id] = True 
                    #setting the depth of the state by taking the depth of its parent state and increasing it by 1
                    depth[s_id] = depth[current] + 1
        
        #appended the visited states to the visited array
        for i in range(len(visited)):
            if visited[i] == True:
                self.visited.append(self.states[i].id)
        
        #displaying the visited array
        print("Visited array:",self.visited)
        #getting the maximum depth from the depth array 
        max_depth = max(depth.values())
        #returning the maximum depth
        return max_depth
    
    def calculate_depth(self):
        #calling the breadth first search function to get the depth
        depth = self.breadth_first_search()
        #displaying the depth
        print("Depth: ", depth, "\n")
    
    def hopcrofts_algorithm(self):

        final_states = []
        not_final_states = []
        
        #looping through the final and visited arrays
        for state in self.final:
            if state in self.visited:
                #adding only the reachable final states
                final_states.append(state)
        
        #looping through the not final and visited arrays
        for state in self.not_final:
            if state in self.visited:
                #adding only the reachable final states
                not_final_states.append(state)
        
        #setting the partition to contain the final reachable states and not final reachable states
        P = [final_states, not_final_states]
        #setting the waitlist to contain the final reachable states and not final reachable states
        W = [final_states, not_final_states]  
        
        while len(W) != 0:
            A = W.pop(0)
            for c in self.alphabet:
                X = []
                for s_id in self.visited: 
                    if (self.states[s_id].transitions[c] in A) and (self.states[s_id].transitions[c] not in X):
                            #appending entire state into X
                            X.append(self.states[s_id]) 

                for Y in P:
                    #computing the intersection and difference
                    inter = (set(X).intersection(Y))
                    diff = (set(Y).difference(X))
                    
                    #if intersection and difference are not empty
                    if inter and diff:
                        #removing the current part from the partitions and adding the intersection and difference
                        P.remove(Y)
                        P.append(inter)
                        P.append(diff)
                        if Y in W:
                            #removing the current part from the waitlist and adding the intersection and difference
                            W.remove(Y)
                            W.append(inter)
                            W.append(diff)
                        #otherwise, either the intersection or the difference will be appended to the waitlist
                        elif len(inter) <= len(diff):
                            W.append(inter)
                        else:
                            W.append(diff)
        
        #returning the partitions
        return P

    def minimise(self):
        #calling the hopcrofts algorithm function to get the partitions
        P = self.hopcrofts_algorithm()
        
        new_states = []
        new_final = []
        new_not_final = []
        new_initial = -1
        
        #looping through the partitions
        for i in range(len(P)):
            #taking the first state of each partition to check the transitions and if it is a final state or not
            state_id = P[i][0]
            state = self.states[state_id]
            #setting the partition as a state
            new_partition_state = State(s_id=i, accepting = state.accepting)
            
            #looping through the alphabet to set the transitions of the partition
            for letter in self.alphabet:
                s_id = state.transitions[letter]
                #looping through all the partitions values
                for x in range(len(P)):
                    for y in range(len(P[i])):
                        #if state id is stored in partitions, then it will be set to the transitions
                        if s_id in P[x]:
                            new_partition_state.transitions[letter] = x

            if self.initial in P[i]:
                #setting the partition which contains the initial state as the initial
                initial = i 
            
            #adding the new partition as a state in the states array
            new_states.append(new_partition_state)
            
            #adding the partition id to either the final or not final array
            if new_partition_state.accepting == True:
                new_final.append(i)
            elif new_partition_state.accepting == False:
                new_not_final.append(i)
            
            #storing the partition states in the parition 
            new_partition_state.set_partition_states(P[i])
            
        #getting the number of partitions
        new_n = len(P)
        
        #just in case the initial state hasnt been set
        if new_initial == -1:
            new_initial = random.randint(0, new_n-1) #choosing a random state as the initial state
        
        #creating the new dfsa with the partitions 
        new_dfsa = DFSA(n = new_n, auto_fill = False, initial = new_initial, final = new_final, not_final = new_not_final, states = new_states)

        return new_dfsa
    
class State:

    def __init__(self, s_id, accepting = None):
        #setting the state details
        self.id = s_id
        if accepting == None:
            #fliping a coin to randomly decide whether state is accepting or not
            self.accepting = flip_coin()
        else:
            self.accepting = accepting
        #setting transitions to initially be -1
        self.transitions = {'a': -1, 'b': -1}
        self.partition_states = []
    
    def set_transitions(self, a, b):
        #setting the state transitions
        self.transitions = {'a': a, 'b': b} #a and b are state ids
    
    def set_partition_states(self, states_ids):
        #setting the partition states
        self.partition_states = states_ids
    
    def print_details(self):
        #printing the state details
        print("State ID: ", self.id)
        print("Transition A: state ", self.transitions['a'])
        print("Transition B: state ", self.transitions['b'])
        print("Is accepting: ", self.accepting)
        if self.partition_states:
            print("Partition States: ", self.partition_states)
        print()

    
n = random.randint(16,64) #random number between 16 and 64 inclusive


def flip_coin():
    coin =  bool(random.getrandbits(1)) #either True or False
    return coin
    

A = DFSA(15)
DFSA.print_details(A)
DFSA.calculate_depth(A)
M = DFSA.minimise(A)
DFSA.print_details(M)
DFSA.calculate_depth(M)


Number of states:  15
Alphabet:  ['a', 'b']
Initial state id:  0
Final states array:  [1, 2, 3, 4, 6, 7, 8, 9, 14]
Not final states array:  [0, 5, 10, 11, 12, 13]

State ID:  0
Transition A: state  1
Transition B: state  2
Is accepting:  False

State ID:  1
Transition A: state  3
Transition B: state  11
Is accepting:  True

State ID:  2
Transition A: state  12
Transition B: state  7
Is accepting:  True

State ID:  3
Transition A: state  4
Transition B: state  3
Is accepting:  True

State ID:  4
Transition A: state  14
Transition B: state  13
Is accepting:  True

State ID:  5
Transition A: state  12
Transition B: state  0
Is accepting:  False

State ID:  6
Transition A: state  8
Transition B: state  6
Is accepting:  True

State ID:  7
Transition A: state  12
Transition B: state  11
Is accepting:  True

State ID:  8
Transition A: state  0
Transition B: state  14
Is accepting:  True

State ID:  9
Transition A: state  8
Transition B: state  1
Is accepting:  True

State ID:  10
Transition 